# Statistical Analysis for Demographics

This notebook focuses on the statistical analysis for the results obtained in the Demographic Analysis section of main notebook. 
The main goal is to statistically measure the difference between socio-economics characteristics in terms of given situtation. 
An ANOVA test for equality of means is performed and deeper analysis is made with confidence intervals for the difference between two means.

## Environment Setup

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
import scipy.stats
from statsmodels.formula.api import ols

default_figsize = (12.8, 4.8)

## Data

### Household Demographic

In [4]:
# Household demographics
df_household = pd.read_csv('dunnhumby/hh_demographic.csv', sep = ',')

#### Data Cleaning

In [5]:
# correct dtypes
df_household['MARITAL_STATUS_CODE'] = df_household['MARITAL_STATUS_CODE'].astype('category')
df_household['HOMEOWNER_DESC'] =  df_household['HOMEOWNER_DESC'].astype('category')
df_household['HH_COMP_DESC'] = df_household['HH_COMP_DESC'].astype('category')
df_household['HOUSEHOLD_SIZE_DESC'] = pd.Categorical(df_household['HOUSEHOLD_SIZE_DESC'].values, categories=['1', '2', '3', '4', '5+'], ordered=True)
df_household['KID_CATEGORY_DESC'] = pd.Categorical(df_household['KID_CATEGORY_DESC'].values, categories=['None/Unknown', '1', '2', '3+'], ordered=True)

# merge some income categories
cat_to_combine_under50K = ["Under 15K", "15-24K", "25-34K", "35-49K"]
cat_to_combine_between50K_100K = ["50-74K", "75-99K"]
cat_to_combine_over100K = ["100-124K", "125-149K", "150-174K", "175-199K", "200-249K", "250K+"]
def combine_cat_income(x):
    if x in cat_to_combine_over100K:
        return "100K+"
    elif x in cat_to_combine_between50K_100K:
        return "50-99K"
    elif x in cat_to_combine_under50K:
        return "Under 50K"
    else:
        print(x)
        return 'ERROR'
    
df_household['INCOME_DESC'] = df_household['INCOME_DESC'].apply(combine_cat_income)

# merge some age categories
cat_to_combine_over55 = ["55-64", "65+"]
cat_to_combine_under35 = ["19-24", "25-34"]
def combine_cat_age(x):
    if x in cat_to_combine_over55:
        return "55+" 
    elif x in cat_to_combine_under35:
        return "19-34"
    else: 
        return x
    
df_household['AGE_DESC'] = df_household['AGE_DESC'].apply(combine_cat_age)


# correct dtypes
df_household['INCOME_DESC'] = pd.Categorical(df_household['INCOME_DESC'].values, categories=['Under 50K', '50-99K', '100K+'], ordered=True)
df_household['AGE_DESC'] = pd.Categorical(df_household['AGE_DESC'].values, categories=['19-34', '35-44', '45-54', '55+'], ordered=True)

### Products

In [6]:
df_products = pd.read_csv('dunnhumby/product.csv', sep = ',')
df_food_products = pd.read_csv('data/food_products.csv')

df_meat_products = pd.read_csv('data/meat_products.csv')
df_veggies_products = pd.read_csv('data/vegetables_products.csv')

df_pure_meat = pd.read_csv("data/pure_meat_products.csv")
df_red_meat = df_pure_meat[df_pure_meat.MEAT_TYPE.isin(['BOVINE', 'OVINE','PORK'])]
df_white_meat = df_pure_meat[df_pure_meat.MEAT_TYPE.isin(['POULTRY'])]
df_sea_food = df_pure_meat[df_pure_meat.MEAT_TYPE.isin(['SEAFOOD'])]

### Transactions

In [7]:
df_transactions = pd.read_csv("data/clean_transactions.csv")
df_food_transactions = pd.read_csv("data/food_transactions.csv")

df_meat_transactions = pd.merge(df_meat_products, df_food_transactions, on="PRODUCT_ID", how="inner")
df_veggies_transactions = pd.merge(df_veggies_products, df_food_transactions, on="PRODUCT_ID", how="inner")

df_pure_meat_transactions = pd.merge(df_pure_meat, df_food_transactions, on="PRODUCT_ID", how="inner")
df_red_meat_transactions = pd.merge(df_red_meat, df_food_transactions, on="PRODUCT_ID", how="inner")
df_white_meat_transactions = pd.merge(df_white_meat, df_food_transactions, on="PRODUCT_ID", how="inner")
df_sea_food_transactions = pd.merge(df_sea_food, df_food_transactions, on="PRODUCT_ID", how="inner")

### Aggregated Dataset

In [8]:
# find active weeks  and total spendings per household
weeks_activity_by_household = df_food_transactions.groupby("household_key").agg({"WEEK_NO": set, "PAID_PRICE": sum})
# create new columns
weeks_activity_by_household["number_of_active_weeks"] = weeks_activity_by_household["WEEK_NO"].apply(lambda x: len(x))
weeks_activity_by_household["avg_spending_per_week"] = weeks_activity_by_household["PAID_PRICE"] / weeks_activity_by_household["number_of_active_weeks"]
# drop week_no and sort
weeks_activity_by_household = weeks_activity_by_household.drop(columns="WEEK_NO")
weeks_activity_by_household = weeks_activity_by_household.sort_values(by="number_of_active_weeks", ascending=False).rename(columns={"PAID_PRICE": "PAID_FOOD"})

In [9]:
# find active weeks  and total spendings per household
weeks_activity_by_household = df_food_transactions.groupby("household_key").agg({"WEEK_NO": set, "PAID_PRICE": sum})
# create new columns
weeks_activity_by_household["number_of_active_weeks"] = weeks_activity_by_household["WEEK_NO"].apply(lambda x: len(x))
weeks_activity_by_household["avg_spending_per_week"] = weeks_activity_by_household["PAID_PRICE"] / weeks_activity_by_household["number_of_active_weeks"]
# drop week_no and sort
weeks_activity_by_household = weeks_activity_by_household.drop(columns="WEEK_NO")
weeks_activity_by_household = weeks_activity_by_household.sort_values(by="number_of_active_weeks", ascending=False).rename(columns={"PAID_PRICE": "PAID_FOOD"})

# add more info 
dfs = [df_meat_transactions, df_veggies_transactions, df_pure_meat_transactions, df_red_meat_transactions, df_white_meat_transactions, df_sea_food_transactions]
df_names = ['MEAT', 'VEGGIE', 'PURE_MEAT', 'RED_MEAT', 'WHITE_MEAT', 'SEA_FOOD']

spent_by_household = weeks_activity_by_household.rename(columns={"PAID_PRICE_x": "paid_FOOD", "PAID_PRICE_y": "paid"})
for name, df in zip(df_names, dfs):
    spending = df.groupby("household_key").agg({"PAID_PRICE": sum}).reset_index().rename(columns={"PAID_PRICE": "PAID_" + name})
    spent_by_household = pd.merge(spending, spent_by_household, on="household_key")
    
    spent_by_household['avg_spending_per_week_' + name] = spent_by_household['PAID_' + name] / spent_by_household['number_of_active_weeks']
    spent_by_household['avg_spending_ratio_' + name] = spent_by_household['avg_spending_per_week_' + name] / spent_by_household['avg_spending_per_week']


## Statistical Analysis for Demographical comparison

The ANOVA tests are shown for all the plots in the Demographical Analysis section of the main notebook. Comments on the results can be found in that notebook.

In [10]:
df_spent_and_demog=pd.merge(df_household, spent_by_household, on='household_key', how='inner')

In [11]:
def stats_data(df, demographic_feature, order=None):
    COLS_OF_INTEREST = [
    "avg_spending_per_week",  
    "avg_spending_ratio_MEAT",
    "avg_spending_ratio_VEGGIE",
    "avg_spending_ratio_PURE_MEAT",
    "avg_spending_ratio_RED_MEAT", 
    "avg_spending_ratio_WHITE_MEAT",
    "avg_spending_ratio_SEA_FOOD",
    ]
    
    
    for i, col in enumerate(COLS_OF_INTEREST):    
        print(demographic_feature, col)
df_spent_and_demog.columns    

Index(['AGE_DESC', 'MARITAL_STATUS_CODE', 'INCOME_DESC', 'HOMEOWNER_DESC',
       'HH_COMP_DESC', 'HOUSEHOLD_SIZE_DESC', 'KID_CATEGORY_DESC',
       'household_key', 'PAID_SEA_FOOD', 'PAID_WHITE_MEAT', 'PAID_RED_MEAT',
       'PAID_PURE_MEAT', 'PAID_VEGGIE', 'PAID_MEAT', 'PAID_FOOD',
       'number_of_active_weeks', 'avg_spending_per_week',
       'avg_spending_per_week_MEAT', 'avg_spending_ratio_MEAT',
       'avg_spending_per_week_VEGGIE', 'avg_spending_ratio_VEGGIE',
       'avg_spending_per_week_PURE_MEAT', 'avg_spending_ratio_PURE_MEAT',
       'avg_spending_per_week_RED_MEAT', 'avg_spending_ratio_RED_MEAT',
       'avg_spending_per_week_WHITE_MEAT', 'avg_spending_ratio_WHITE_MEAT',
       'avg_spending_per_week_SEA_FOOD', 'avg_spending_ratio_SEA_FOOD'],
      dtype='object')

### AGE_DESC

In [12]:
results = ols('avg_spending_per_week ~ C(AGE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_MEAT ~ C(AGE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_VEGGIE ~ C(AGE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_PURE_MEAT ~ C(AGE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_RED_MEAT ~ C(AGE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_WHITE_MEAT ~ C(AGE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_SEA_FOOD ~ C(AGE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     avg_spending_per_week   R-squared:                       0.021
Model:                               OLS   Adj. R-squared:                  0.017
Method:                    Least Squares   F-statistic:                     5.303
Date:                   Sat, 14 Dec 2019   Prob (F-statistic):            0.00128
Time:                           20:28:00   Log-Likelihood:                -3392.9
No. Observations:                    734   AIC:                             6794.
Df Residuals:                        730   BIC:                             6812.
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
In

### INCOME_DESC

In [13]:
results = ols('avg_spending_per_week ~ C(INCOME_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_MEAT ~ C(INCOME_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_VEGGIE ~ C(INCOME_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_PURE_MEAT ~ C(INCOME_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_RED_MEAT ~ C(INCOME_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_WHITE_MEAT ~ C(INCOME_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_SEA_FOOD ~ C(INCOME_DESC)', data = df_spent_and_demog).fit()
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     avg_spending_per_week   R-squared:                       0.099
Model:                               OLS   Adj. R-squared:                  0.096
Method:                    Least Squares   F-statistic:                     40.12
Date:                   Sat, 14 Dec 2019   Prob (F-statistic):           2.93e-17
Time:                           20:28:16   Log-Likelihood:                -3362.6
No. Observations:                    734   AIC:                             6731.
Df Residuals:                        731   BIC:                             6745.
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

### HOUSEHOLD_SIZE_DESC

In [16]:
results = ols('avg_spending_per_week ~ C(HOUSEHOLD_SIZE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_MEAT ~ C(HOUSEHOLD_SIZE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_VEGGIE ~ C(HOUSEHOLD_SIZE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_PURE_MEAT ~ C(HOUSEHOLD_SIZE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_RED_MEAT ~ C(HOUSEHOLD_SIZE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_WHITE_MEAT ~ C(HOUSEHOLD_SIZE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_SEA_FOOD ~ C(HOUSEHOLD_SIZE_DESC)', data = df_spent_and_demog).fit()
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     avg_spending_per_week   R-squared:                       0.034
Model:                               OLS   Adj. R-squared:                  0.029
Method:                    Least Squares   F-statistic:                     6.477
Date:                   Sat, 14 Dec 2019   Prob (F-statistic):           4.00e-05
Time:                           20:28:44   Log-Likelihood:                -3388.0
No. Observations:                    734   AIC:                             6786.
Df Residuals:                        729   BIC:                             6809.
Df Model:                              4                                         
Covariance Type:               nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

### HH_COMP_DESC

In [17]:
results = ols('avg_spending_per_week ~ C(HH_COMP_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_MEAT ~ C(HH_COMP_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_VEGGIE ~ C(HH_COMP_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_PURE_MEAT ~ C(HH_COMP_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_RED_MEAT ~ C(HH_COMP_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_WHITE_MEAT ~ C(HH_COMP_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_SEA_FOOD ~ C(HH_COMP_DESC)', data = df_spent_and_demog).fit()
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     avg_spending_per_week   R-squared:                       0.035
Model:                               OLS   Adj. R-squared:                  0.028
Method:                    Least Squares   F-statistic:                     5.243
Date:                   Sat, 14 Dec 2019   Prob (F-statistic):           9.73e-05
Time:                           20:29:00   Log-Likelihood:                -3387.9
No. Observations:                    734   AIC:                             6788.
Df Residuals:                        728   BIC:                             6815.
Df Model:                              5                                         
Covariance Type:               nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

### KID_CATEGORY_DESC

In [18]:
results = ols('avg_spending_per_week ~ C(KID_CATEGORY_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_MEAT ~ C(KID_CATEGORY_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_VEGGIE ~ C(KID_CATEGORY_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_PURE_MEAT ~ C(KID_CATEGORY_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_RED_MEAT ~ C(KID_CATEGORY_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_WHITE_MEAT ~ C(KID_CATEGORY_DESC)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_SEA_FOOD ~ C(KID_CATEGORY_DESC)', data = df_spent_and_demog).fit()
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     avg_spending_per_week   R-squared:                       0.032
Model:                               OLS   Adj. R-squared:                  0.028
Method:                    Least Squares   F-statistic:                     7.918
Date:                   Sat, 14 Dec 2019   Prob (F-statistic):           3.34e-05
Time:                           20:29:32   Log-Likelihood:                -3389.1
No. Observations:                    734   AIC:                             6786.
Df Residuals:                        730   BIC:                             6805.
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

### MARITAL_STATUS_CODE

In [20]:
results = ols('avg_spending_per_week ~ C(MARITAL_STATUS_CODE)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_MEAT ~ C(MARITAL_STATUS_CODE)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_VEGGIE ~ C(MARITAL_STATUS_CODE)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_PURE_MEAT ~ C(MARITAL_STATUS_CODE)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_RED_MEAT ~ C(MARITAL_STATUS_CODE)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_WHITE_MEAT ~ C(MARITAL_STATUS_CODE)', data = df_spent_and_demog).fit()
print(results.summary())
results = ols('avg_spending_ratio_SEA_FOOD ~ C(MARITAL_STATUS_CODE)', data = df_spent_and_demog).fit()
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     avg_spending_per_week   R-squared:                       0.017
Model:                               OLS   Adj. R-squared:                  0.014
Method:                    Least Squares   F-statistic:                     6.287
Date:                   Sat, 14 Dec 2019   Prob (F-statistic):            0.00196
Time:                           20:32:11   Log-Likelihood:                -3394.6
No. Observations:                    734   AIC:                             6795.
Df Residuals:                        731   BIC:                             6809.
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------